In [6]:
from flask import Flask, render_template, request
from litellm import completion
import docx

In [7]:
from langchain.llms import Ollama
ollama = Ollama(base_url='http://localhost:11434',
model="llama2")

In [8]:
def read_docx(file):
    doc = docx.Document(file)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

In [35]:
app = Flask(__name__)


@app.route('/')
def index():
    return render_template('index.html')

@app.route('/ask_question', methods=['POST'])
def ask_question():
    question = request.form['question']
    file_paths = request.files.getlist('files')
    responses = []
    
    for file in file_paths:
        if file.filename.endswith('.txt'):
            content = file.read().decode('utf-8', 'ignore')
        elif file.filename.endswith('.docx'):
            content = read_docx(file)
        else:
            return render_template('error.html', message="Unsupported file format")

        try:
            response = completion(model="ollama/llama2", messages=[{"content": content, "role": "user"},{"content": question, "role": "user"}])
            response_text = response.choices[0].message.content
            responses.append((file.filename, response_text))
        except Exception as e:
            error_message = f"Error processing file {file.filename}: {str(e)}"
            responses.append((file.filename, error_message))

    
    return render_template('responses.html', question=question, responses=responses)

if __name__ == "__main__":
    app.run(host='127.0.0.1', port=5002)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5002
Press CTRL+C to quit
127.0.0.1 - - [11/Mar/2024 13:54:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2024 13:54:10] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Mar/2024 13:54:44] "POST /ask_question HTTP/1.1" 200 -


In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("C:/Users/a-elearoche/Documents/Continuum/Transcriptions/Transcription_test2_2024-02-15.pdf")
pages = loader.load_and_split()

In [ ]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="llama2")
datastore_directory = '../.datastore'
vectorstore = Chroma.from_documents(documents=pages, embedding=oembed, persist_directory = datastore_directory)
vectorstore.persist()

In [ ]:
question="Quelle est la premiere phrase du texte ? et la derniere ?"
docs = vectorstore.similarity_search(question)
len(docs)

In [ ]:
from langchain.chains import RetrievalQA
qachain=RetrievalQA.from_chain_type(ollama, retriever=vectorstore.as_retriever())
qachain({"query": question})